~~~
Copyright 2025 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
~~~

# Quick start with Hugging Face

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/google-gemini/gemma-cookbook/blob/main/TxGemma/[TxGemma]Quickstart_with_Hugging_Face.ipynb">
      <img alt="Google Colab logo" src="https://www.tensorflow.org/images/colab_logo_32px.png" width="32px"><br> Run in Google Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/google-gemini/gemma-cookbook/blob/main/TxGemma/%5BTxGemma%5DQuickstart_with_Hugging_Face.ipynb">
      <img alt="GitHub logo" src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" width="32px"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://huggingface.co/collections/google/txgemma-release-67dd92e931c857d15e4d1e87">
      <img alt="HuggingFace logo" src="https://huggingface.co/front/assets/huggingface_logo-noborder.svg" width="32px"><br> View on HuggingFace
    </a>
  </td>
</tr></tbody></table>

This notebook provides a basic demo of using TxGemma, a collection of large language models built upon Gemma 2, that generates predictions, classifications or text based on therapeutic related data. It contains standalone usage examples for both:

- Predictive tasks, which expect a narrow form of prompting
- Conversational use (for TxGemma-Chat model variants), which is more flexible and includes multi-turn interactions

Learn more about the model at [this page](https://developers.google.com/health-ai-developer-foundations/txgemma).

## Setup

To complete this tutorial, you'll need to have a Colab runtime with sufficient resources to run the TxGemma model. Choose an appropriate runtime when starting your Colab session.

You can try out TxGemma 2B or 9B* for free using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

*To run the demo with TxGemma 9B on a T4 GPU, use int8 quantization to reduce memory usage and speed up inference. Note that the performance of quantized versions has not been evaluated.

### Get access to TxGemma

Before you get started, make sure that you have access to TxGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [TxGemma model page](https://huggingface.co/google/txgemma-2b-predict) and accept the usage conditions.

### Configure your HF token

Generate a Hugging Face `read` access token by clicking [here](https://huggingface.co/settings/tokens) and add your access token to the Colab Secrets manager to securely store it.

1. Open your Google Colab notebook and click on the 🔑 Secrets tab in the left panel. <img src="https://storage.googleapis.com/generativeai-downloads/images/secrets.jpg" alt="The Secrets tab is found on the left panel." width=50%>
2. Create a new secret with the name `HF_TOKEN`.
3. Copy/paste your token key into the Value input box of `HF_TOKEN`.
4. Toggle the button on the left to allow notebook access to the secret.

In [1]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")

### Install dependencies

In [2]:

! pip install --upgrade --quiet accelerate bitsandbytes huggingface_hub transformers

## Load model from Hugging Face Hub

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_VARIANT = "9b-chat"  # @param ["2b-predict", "9b-chat", "9b-predict", "27b-chat", "27b-predict"]

model_id = f"google/txgemma-{MODEL_VARIANT}"

if MODEL_VARIANT == "2b-predict":
    additional_args = {}
else:
    additional_args = {
        "quantization_config": BitsAndBytesConfig(load_in_8bit=True)
    }

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    **additional_args,
)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

After loading, you can directly use the model and tokenizer, which gives you complete control over the inference process, including tokenization and postprocessing of outputs.

Alternatively, you can use the [`pipeline`](https://www.google.com/url?q=https%3A%2F%2Fhuggingface.co%2Fdocs%2Ftransformers%2Fen%2Fmain_classes%2Fpipelines) API, which provides a simple way to use the model for inference while abstracting away complex details. Here, instantiate a text generation pipeline using the loaded model:

In [4]:
from transformers import pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

Device set to use cuda:0


The following sections include standalone examples demonstating how to use the model both directly and with the `pipeline` API. In practice, you should select the method that is best suited for your use case.

## Format prompts for therapeutic tasks

The following sections in this notebook demonstrate prompting TxGemma for therapeutic development tasks from [Therapeutics Data Commons](https://tdcommons.ai/) (TDC).

For these predictive tasks, prompts should be formatted according to the TDC structure, including:

- **Instruction:** Briefly describes the task.

- **Context:** Provides 2-3 sentences of relevant biochemical background, derived from TDC descriptions and literature.

- **Question:** Queries a specific therapeutic property, incorporating textual representations of therapeutics and/or targets as inputs.

  - Inputs can include SMILES strings, amino acid sequences, nucleotide sequences, and natural language text.

  - Optional few-shot examples can be provided.

### Load prompt template

First, load a JSON file that contains the prompt format for various TDC tasks.

In [5]:
import json
from huggingface_hub import hf_hub_download

tdc_prompts_filepath = hf_hub_download(
    repo_id=model_id,
    filename="tdc_prompts.json",
)

with open(tdc_prompts_filepath, "r") as f:
    tdc_prompts_json = json.load(f)

tdc_prompts.json:   0%|          | 0.00/768k [00:00<?, ?B/s]

### Prepare sample prompt

Construct a prompt using the template and an input drug SMILES string from the [BBB (Blood-Brain Barrier), Martins et al.](https://tdcommons.ai/single_pred_tasks/adme#bbb-blood-brain-barrier-martins-et-al) dataset. This prompt will be used for generating predictions in the next sections.

In [6]:
# Set example task and input
task_name = "BBB_Martins"
input_type = "{Drug SMILES}"
drug_smiles = "CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21"

TDC_PROMPT = tdc_prompts_json[task_name].replace(input_type, drug_smiles)
print("Formatted prompt:\n")
print(TDC_PROMPT)

Formatted prompt:

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:


## Explore predictive capabilities

TxGemma models are designed to process and understand information related to various therapeutic modalities and targets, including small molecules, proteins, nucleic acids, diseases, and cell lines, and can generate predictions on a broad set of therapeutic development tasks.

### Run inference on a therapeutic task

This section demonstrates prompting TxGemma for a predictive task from TDC.


**Run the model directly**

In [7]:
# Use sample prompt for a predictive task from TDC
prompt = TDC_PROMPT

# Prepare tokenized inputs
input_ids = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(**input_ids, max_new_tokens=8)
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer: (B)


**Run with the `pipeline` API**

In [8]:
# Use sample prompt for a predictive task from TDC
prompt = TDC_PROMPT

# Generate response
outputs = pipe(prompt, max_new_tokens=8)
response = outputs[0]["generated_text"]
print(response)

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer: (B)


## Explore conversational capabilities with TxGemma-Chat

TxGemma features conversational models that add reasoning and explainability to predictions and can be used in multi-turn interactions. Their conversational ability comes at the expense of some predictive power.

**For this section, make sure that you have selected a TxGemma-Chat model variant.**

### Ask questions in a multi-turn conversation

This section demonstrates prompting TxGemma for conversational use, adhering to the [Gemma chat template](https://ai.google.dev/gemma/docs/core/prompt-structure).

In this example, first prompt the model to answer a question regarding a predictive task using the TDC format. Then, ask a follow-up question requesting the model to provide its reasoning for the predicted answer.

**Run the model directly**

In [9]:
from IPython.display import display, Markdown

questions = [
    TDC_PROMPT,  # Initial question is a predictive task from TDC
    "Explain your reasoning based on the molecule structure."
]

messages = []

display(Markdown("\n\n---\n\n"))
for question in questions:
    display(Markdown(f"**User:**\n\n{question}\n\n---\n\n"))
    messages.append(
        { "role": "user", "content": question },
    )
    # Apply the tokenizer's built-in chat template
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to("cuda"), max_new_tokens=512)
    response = tokenizer.decode(outputs[0, len(inputs[0]):], skip_special_tokens=True)
    display(Markdown(f"**TxGemma:**\n\n{response}\n\n---\n\n"))
    messages.append(
        { "role": "assistant", "content": response},
    )



---



**User:**

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:

---



**TxGemma:**

(B)

---



**User:**

Explain your reasoning based on the molecule structure.

---



**TxGemma:**

Here's the breakdown of why the drug with SMILES CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21 likely crosses the BBB:

* **Small Size:** The molecule is relatively small, which is a general favorable characteristic for BBB penetration. Larger molecules have a harder time squeezing through the tight junctions between brain endothelial cells.
* **Lipophilicity (Hydrophobicity):** The presence of multiple carbon and hydrogen atoms in the benzene rings and alkyl chains makes the molecule predominantly lipophilic (fat-loving).  Lipophilicity is a key determinant of BBB permeability. The more lipophilic a drug, the easier it crosses the lipid-rich cell membranes of the BBB.
* **Lack of Charged Groups:**  The molecule lacks large, charged groups (like carboxyl or amine groups).  Charged groups tend to be repelled by the lipid bilayer of the BBB, making it harder to cross. 
* **Absence of Specific BBB Targets:** While some drugs have specific transporters that help them across the BBB, this molecule doesn't appear to have any obvious targeting groups.

**In summary, the drug's small size, lipophilic nature, lack of significant charge, and absence of specific BBB-targeting groups suggest that it likely possesses a good ability to cross the blood-brain barrier.**

**Important Note:** This is a general prediction based on structural features. Actual BBB permeability is complex and can be influenced by many factors. Experimental validation is always necessary to confirm drug penetration ability. 


---



**Run with the `pipeline` API**

In [10]:
from IPython.display import display, Markdown

questions = [
    TDC_PROMPT,  # Initial question is a predictive task from TDC
    "Explain your reasoning based on the molecule structure."
]

messages = []

display(Markdown("\n\n---\n\n"))
for question in questions:
    display(Markdown(f"**User:**\n\n{question}\n\n---\n\n"))
    messages.append(
        { "role": "user", "content": question },
    )
    outputs = pipe(messages, max_new_tokens=512)
    messages = outputs[0]["generated_text"]
    response = messages[-1]["content"].strip()
    display(Markdown(f"**TxGemma:**\n\n{response}\n\n---\n\n"))



---



**User:**

Instructions: Answer the following question about drug properties.
Context: As a membrane separating circulating blood and brain extracellular fluid, the blood-brain barrier (BBB) is the protection layer that blocks most foreign drugs. Thus the ability of a drug to penetrate the barrier to deliver to the site of action forms a crucial challenge in development of drugs for central nervous system.
Question: Given a drug SMILES string, predict whether it
(A) does not cross the BBB (B) crosses the BBB
Drug SMILES: CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21
Answer:

---



**TxGemma:**

(B)

---



**User:**

Explain your reasoning based on the molecule structure.

---



**TxGemma:**

Here's the breakdown of why the drug with SMILES CN1C(=O)CN=C(C2=CCCCC2)c2cc(Cl)ccc21 likely crosses the BBB:

* **Small Size:** The molecule is relatively small, which is a general favorable characteristic for BBB penetration. Larger molecules have a harder time squeezing through the tight junctions between brain endothelial cells.
* **Lipophilicity (Hydrophobicity):** The presence of multiple carbon and hydrogen atoms in the benzene rings and alkyl chains makes the molecule predominantly lipophilic (fat-loving).  Lipophilicity is a key determinant of BBB permeability. The more lipophilic a drug, the easier it crosses the lipid-rich cell membranes of the BBB.
* **Lack of Charged Groups:**  The molecule lacks large, charged groups (like carboxyl or amine groups).  Charged groups tend to be repelled by the lipid bilayer of the BBB, making it harder to cross. 
* **Absence of Specific BBB Targets:** While some drugs have specific transporters that help them across the BBB, this molecule doesn't appear to have any obvious targeting groups.

**In summary, the drug's small size, lipophilic nature, lack of significant charge, and absence of specific BBB-targeting groups suggest that it likely possesses a good ability to cross the blood-brain barrier.**

**Important Note:** This is a general prediction based on structural features. Actual BBB permeability is complex and can be influenced by many factors. Experimental validation is always necessary to confirm drug penetration ability.

---



# Next steps

Explore the other [notebooks](https://github.com/google-gemini/gemma-cookbook/blob/main/TxGemma) to learn what else you can do with the model.